# Домашнее задание № 10. Генерация текста

### Задание 1 (8 баллов).

Попробуйте дообучать GPT на каком-то другом тексте (можете попробовать любые стихи или какие-то специфичные вещи вроде анекдотов, теорий заговоров, постов в помоечных телеграм каналах, текстов журналистов и СМИ с выразительным стилем). 
Попробуйте разные методы и параметры генерации (beam search, температура, top_k и тп). Сохраните в тетрадке несколько хороших сгенерированных текстов


In [ ]:
!pip install transformers

     |████████████████████████████████| 4.0 MB 5.1 MB/s 
     |████████████████████████████████| 6.6 MB 31.2 MB/s 
     |████████████████████████████████| 596 kB 35.4 MB/s 
     |████████████████████████████████| 880 kB 30.6 MB/s 
     |████████████████████████████████| 77 kB 5.4 MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895260 sha256=551550adccb4e5ca44a1744fde6634fda6b967a979b19c0aa1e17433286d4080
  Stored in directory: /root/.cache/pip/wheels/87/39/dd/a83eeef36d0bf98e7a4d1933a4ad2d660295a40613079bafc9
Successfully built sacremoses
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import torch

DEVICE = torch.device("cuda:0")

# возьмем модель поменьше, так как дообучение это обновление весов
model_name_or_path = "sberbank-ai/rugpt3small_based_on_gpt2"
tokenizer = GPT2Tokenizer.from_pretrained(model_name_or_path)
model = GPT2LMHeadModel.from_pretrained(model_name_or_path, use_cache=False).to(DEVICE)

Downloading:   0%|          | 0.00/1.63M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.21M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/608 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/526M [00:00<?, ?B/s]

In [ ]:
!pip install lyricsgenius

     |████████████████████████████████| 59 kB 3.2 MB/s 


Можно попытаться дообучить модель на песнях разных жанров и посмотреть, что из этого получится. Для этого нужно откуда-то вытащить тексты песен (есть библиотека для genius) и дообучить их на модели.

In [ ]:
token = 'yCIuYMibeTBLWLEVMEBgh2v1T03npyBiER4FoUGUS6VT9_lPJMl9rI7aU3qv8AtM'

In [ ]:
from lyricsgenius import Genius

search_list = ['Свидетельство о Смерти', 'Мать Тереза', 'Чёрный Лукич', 'Спинки Мента', 'Звуки Му', 'Гражданская оборона', 'Слава КПСС', 'Замай', 'MORGENSHTERN']

genius = Genius(token)
bands = []
for search in search_list:
  bands.append(genius.search_artist(search, max_songs=30))

Searching for songs by Свидетельство о Смерти...

Changing artist name to 'Свидетельство О Смерти (Death Certificate)'
Song 1: "Петля (Loop)"
Song 2: "Обосранный герой (Obosranniy geroy)"
Song 3: "Когда погаснет солнце (When the sun goes out)"
Song 4: "Привет (Hello)"
Song 5: "Собаке собачья смерть (Sobake sobachya smert)"
Song 6: "Подводная лодка (Podvodnaya lodka)"
Song 7: "Всегда живой (Always alive)"
Song 8: "Инструкция (Instruction)"
Song 9: "Моё имя (My name)"
Song 10: "Итог любви (Itog lybvi)"
Song 11: "Едет крыша (Roof rides)"
Song 12: "Только не я (Tolko ne ya)"
Song 13: "Оптимистическое (Optimistic)"
Song 14: "Сказки (Skazki)"
Song 15: "Когда я ответил да (Kogda ya otvetil da)"
Song 16: "Женщины боятся (Women are afraid)"
Song 17: "Среди мёртвых (Sredy mertvih)"
Song 18: "Детский сон (Detskiy son)"
Song 19: "Считалочка (Shitalochka)"
Song 20: "Не положено (Ne polozheno)"
Song 21: "Зачем ей (Zachem ey)"
Song 22: "Чуть-чуть меня"
Done. Found 22 songs.
Searching for songs by Мат

In [ ]:
text = ''
for band in bands:
  for song in band.songs:
    text += song.lyrics + "\n\n\n\n\n"

In [ ]:
from transformers import TextDataset, DataCollatorForLanguageModeling

# Сохраним обучающие данные в .txt файл 
train_path = 'train_dataset.txt'
with open(train_path, "w") as f:
    f.write(text)

# Создание датасета
train_dataset = TextDataset( tokenizer=tokenizer,file_path=train_path,block_size=64, 
                            overwrite_cache=True)
  
# специальный класс который будет подавать в модель данные в нужном ей виде
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

/usr/local/lib/python3.7/dist-packages/transformers/data/datasets/language_modeling.py:58: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  FutureWarning,


In [ ]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments( 
    output_dir= "./finetuned",
    overwrite_output_dir=True,
    num_train_epochs=100, 
    per_device_train_batch_size=64, 
    per_device_eval_batch_size=64,  
    gradient_accumulation_steps=32, 
    )


trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    optimizers = (torch.optim.AdamW(model.parameters(),lr=1e-5),None) # Optimizer and lr scheduler
)

In [ ]:
trainer.train()

***** Running training *****
  Num examples = 1877
  Num Epochs = 100
  Instantaneous batch size per device = 64
  Total train batch size (w. parallel, distributed & accumulation) = 2048
  Gradient Accumulation steps = 32
  Total optimization steps = 100


Step,Training Loss




Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=100, training_loss=3.299241027832031, metrics={'train_runtime': 5609.6731, 'train_samples_per_second': 33.46, 'train_steps_per_second': 0.018, 'total_flos': 6130564300800000.0, 'train_loss': 3.299241027832031, 'epoch': 100.0})

In [ ]:
text = "Нас найдут по клочьям плоти "
input_ids = tokenizer.encode(text, return_tensors="pt").to(DEVICE)
model.eval()
with torch.no_grad():
    out = model.generate(input_ids, 
                        do_sample=True,
                        temperature=1.3,
                        #num_beams=20,
                        top_k=40,
                        max_length=250,
                        )

generated_text = list(map(tokenizer.decode, out))[0]
print()
print(generated_text)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



Нас найдут по клочьям плоти 
(Будет всё в этой игре)
Мы не будем думать как жить дальше 
Мы просто умрем!

[Подкаст 1]
Всё в нашем мире - ложь и обман
Всё во всём - иллюзия обмана
Что б хоть как-то выжить, нужен покой
Жаркое лето - на ладони
Её цвет
Одеяло душистых цветов
Отвергнутую жизнь
Что б в груди
Запахло весной не сладко
Но как найти путь живым
Не знаю кто он?
Кто живой?
Он в меня превратился не зря
Я мёртвых чувствую насквозь
Мне нечем помочь ему
Иду за ним, иду за собою
Судьба стучится о моё стекло
Я верю, что если жить, то
Мы добьемся успеха
И тогда мы возьмём новый крест
В моей душе не гаснет
Дружбы снова станет больше
Я найду способ вернуться
Мёртвого друга - живой труп
Смерть, когда же придёт конец
Остаток дня проживу как будто заново

(Текст песни]
Я пришёл в мир смерти

[Куплет 1]
Опавшие звёзды


"Удачные" варианты:

In [2]:
print('''
Дом
Горит в нём огонёк надежды, а я живу другой жизнью
Но как можно жить без войны? Как можно жить без людей?
Да, но как можно жить без воды? Как жить без хлеба? Без солнца?
Как же можно жить без водки? Как можно жить без любви?

[Куплет 3]
Елки-паленки, ты не заметил, как солнце садится нам
Но я с неба не сброшу тебе на голову звёзды, не убьёшь меня ты как герой не убишь
Я люблю тебя сильнее нас всем хватит для любви на двоих
Я люблю твою руку, ты слышишь слова?
Она моя (это ты) как ладан (это ты) как цветок на ветру (это мы), как цветы в саду (это вы)
Я люблю твоё вино… ты веришь в него? 
А-а-а-а, божественный напиток, ты веришь в слово «лёд»?
Ты веришь так, как я?
Ещё скажи мне про этот дождь или про то теплое море…
Я буду ждать твой вечер… как ты… я буду ждать твой свет… как он… как он…
Гайдамаков, ты не заметил,
''')


Дом
Горит в нём огонёк надежды, а я живу другой жизнью
Но как можно жить без войны? Как можно жить без людей?
Да, но как можно жить без воды? Как жить без хлеба? Без солнца?
Как же можно жить без водки? Как можно жить без любви?

[Куплет 3]
Елки-паленки, ты не заметил, как солнце садится нам
Но я с неба не сброшу тебе на голову звёзды, не убьёшь меня ты как герой не убишь
Я люблю тебя сильнее нас всем хватит для любви на двоих
Я люблю твою руку, ты слышишь слова?
Она моя (это ты) как ладан (это ты) как цветок на ветру (это мы), как цветы в саду (это вы)
Я люблю твоё вино… ты веришь в него? 
А-а-а-а, божественный напиток, ты веришь в слово «лёд»?
Ты веришь так, как я?
Ещё скажи мне про этот дождь или про то теплое море…
Я буду ждать твой вечер… как ты… я буду ждать твой свет… как он… как он…
Гайдамаков, ты не заметил,



In [3]:
print('''
Стою в будке, весь мокрый, слишком жарко 
Чёрт знает на сколько, но не холодно!

[Куплет 26]
Я вот только недавно понял, в кого превратил меня
Я сам превратился в стаю бездомных бомжей
Я сам себя превратил в стадо беспутных овец
Негодяи меня взяли под крыло
Только что навестил дом и на душе стало так легко
Но зачем это, ведь вокруг люди уже так устали
В моём мире нет ничего особенного
Я сам стал стал рабом этой стаи
Вот эта волна пошла по моему дому и пошла по нам всем
Ещё я знаю по своему телу, как я это делаю
Все эти тонны бумаги, эти коробки…
Вот и я не стал бы никому мешать здесь (Говорить)
Я сам превратился в стадо беспутных овец
Всего за полчаса (Пауза) я превратил себя в стаю безумств (Параллак)
Я сам превратил себя в стаю безумств (Параллак)
Я сам превратил себя в стаю безумств (Прайш)

[Куплет 30]
Оно мне надо?
Мне надо?
И пусть
''')


Стою в будке, весь мокрый, слишком жарко 
Чёрт знает на сколько, но не холодно!

[Куплет 26]
Я вот только недавно понял, в кого превратил меня
Я сам превратился в стаю бездомных бомжей
Я сам себя превратил в стадо беспутных овец
Негодяи меня взяли под крыло
Только что навестил дом и на душе стало так легко
Но зачем это, ведь вокруг люди уже так устали
В моём мире нет ничего особенного
Я сам стал стал рабом этой стаи
Вот эта волна пошла по моему дому и пошла по нам всем
Ещё я знаю по своему телу, как я это делаю
Все эти тонны бумаги, эти коробки…
Вот и я не стал бы никому мешать здесь (Говорить)
Я сам превратился в стадо беспутных овец
Всего за полчаса (Пауза) я превратил себя в стаю безумств (Параллак)
Я сам превратил себя в стаю безумств (Параллак)
Я сам превратил себя в стаю безумств (Прайш)

[Куплет 30]
Оно мне надо?
Мне надо?
И пусть



In [4]:
print('''
Нас найдут по клочьям плоти 
(Будет всё в этой игре)
Мы не будем думать как жить дальше 
Мы просто умрем!

[Подкаст 1]
Всё в нашем мире - ложь и обман
Всё во всём - иллюзия обмана
Что б хоть как-то выжить, нужен покой
Жаркое лето - на ладони
Её цвет
Одеяло душистых цветов
Отвергнутую жизнь
Что б в груди
Запахло весной не сладко
Но как найти путь живым
Не знаю кто он?
Кто живой?
Он в меня превратился не зря
Я мёртвых чувствую насквозь
Мне нечем помочь ему
Иду за ним, иду за собою
Судьба стучится о моё стекло
Я верю, что если жить, то
Мы добьемся успеха
И тогда мы возьмём новый крест
В моей душе не гаснет
Дружбы снова станет больше
Я найду способ вернуться
Мёртвого друга - живой труп
Смерть, когда же придёт конец
Остаток дня проживу как будто заново

(Текст песни]
Я пришёл в мир смерти

[Куплет 1]
Опавшие звёзды
''')


Нас найдут по клочьям плоти 
(Будет всё в этой игре)
Мы не будем думать как жить дальше 
Мы просто умрем!

[Подкаст 1]
Всё в нашем мире - ложь и обман
Всё во всём - иллюзия обмана
Что б хоть как-то выжить, нужен покой
Жаркое лето - на ладони
Её цвет
Одеяло душистых цветов
Отвергнутую жизнь
Что б в груди
Запахло весной не сладко
Но как найти путь живым
Не знаю кто он?
Кто живой?
Он в меня превратился не зря
Я мёртвых чувствую насквозь
Мне нечем помочь ему
Иду за ним, иду за собою
Судьба стучится о моё стекло
Я верю, что если жить, то
Мы добьемся успеха
И тогда мы возьмём новый крест
В моей душе не гаснет
Дружбы снова станет больше
Я найду способ вернуться
Мёртвого друга - живой труп
Смерть, когда же придёт конец
Остаток дня проживу как будто заново

(Текст песни]
Я пришёл в мир смерти

[Куплет 1]
Опавшие звёзды



### Задание  2 (2 балла)

Ответьте на следующие вопросы:

1) В каких статья были представлены GPT-1, GPT-2, GPT-3?

2) Как собирался обучающий корпус для GPT-3? Каким образом создатели старались обеспечить высокое качество текстов в обучающей выборке?

1) 
GPT-1: https://cdn.openai.com/research-covers/language-unsupervised/language_understanding_paper.pdf, 

GPT-2: https://d4mucfpksywv.cloudfront.net/better-language-models/language_models_are_unsupervised_multitask_learners.pdf, 

GPT-3: https://arxiv.org/pdf/2005.14165.pdf

2) В GPT-3 корпус состоял из статей из википедии, WebText dataset (собранный по ссылкам на веб-страницы), Common Crawl dataset (состоящий из текстов на веб-страницах) и двух непересекающихся корпусов книг. 

Неотфильтрованные данные с Common Crawl dataset давали низкие результаты, по этой причине авторы проделывали три шага к увеличению качества корпуса. На первом шаге они загружали и фильтровали корпус, основываясь на сходстве с ранжированным связанным корпусом высокого качества. Второй шаг заключался в реализации нечёткой дедупликации на уровне документа для предотвращения избыточности и сохранения целостности валидационной выборки в качестве метрики переобучения. Наконец, авторы добавили уже знакомый корпус с высоким качеством в тренировочный набор для увеличения существующего корпуса и улучшения разнообразия данных.